### Load some audio

In [ ]:
import numpy as np

import scipy.io.wavfile
import matplotlib.pyplot as plt

#import sys

In [ ]:
def spectrogram(wav_filepath):
    # https://mail.python.org/pipermail/chicago/2010-December/007314.html
    sample_rate, samples = scipy.io.wavfile.read(wav_filepath)

    # Rescale so that max/min are ~ +/- 1 around 0
    data_av = np.mean(samples)
    data_max = np.max(np.absolute(samples-data_av))
    sound_data = (samples - data_av)/data_max
    
    ## Parameters: 10ms step, 30ms window
    nstep = int(sample_rate * 0.01)
    nwin  = int(sample_rate * 0.03)
    nfft = 2*int(nwin/2)

    window = np.hamming(nwin)

    # will take windows x[n1:n2].  generate and loop over 
    # n2 such that all frames fit within the waveform
    nn = range(nwin, len(sound_data), nstep)

    X = np.zeros( (len(nn), nfft//2) )

    for i,n in enumerate(nn):
        segment = sound_data[ n-nwin:n ]
        z = np.fft.fft(window * segment, nfft)
        X[i,:] = np.log(np.absolute(z[:nfft//2]))
        
    return X

In [ ]:
def smooth(x, window_len=21):  # , window='hanning'
    # http://scipy-cookbook.readthedocs.io/items/SignalSmooth.html
    #s = np.r_[ x[window_len-1:0:-1], x, x[-1:-window_len:-1]]
    s = np.r_[ np.zeros( ((window_len-1)//2,) ), x, np.zeros( ((window_len-1)//2,) ) ]
    w=np.hamming(window_len)
    return np.convolve(w/w.sum(), s, mode='valid') #[window_len-1 : -(window_len-1) ]

In [ ]:
#f = './data/num_phone_en-UK_m_Martin0.wav'
#f = './data/num_Bing_en-UK_f_Susan.wav'

#f = './data/num_phone_en-UK_m_Martin0.ogg'
f = './data/num_Bing_en-UK_f_Susan.ogg'

In [ ]:
X = spectrogram(f)
print("X.shape=", X.shape)

Y = np.std(X, axis=1)
#Y = np.max(X, axis=1)
Y_min = np.min(Y)
Y_range = Y.max()-Y_min
Y = (Y - Y_min)/Y_range

print("Y.shape=", Y.shape)

Y_crop = np.where(Y>0.25, 1.0, 0.0)
# Apply some smoothing

Y_crop = smooth(Y_crop)
Y_crop = np.where(Y_crop>0.01, 1.0, 0.0)
print("Y_crop.shape=", Y_crop.shape)

plt.imshow(X.T, interpolation='nearest',
    origin='lower',
    aspect='auto')

#plt.plot(Y * X.shape[1])

plt.plot(Y_crop * X.shape[1])

plt.show()
#Y.min(), Y.max()
#X[100,:]
np.argmin(X)/248, np.argmax(X)/248

In [ ]:
# Split the file into voiced segments

#X.max()

# Clip off front and back where Y_crop is zero
#http://stackoverflow.com/questions/4494404/find-large-number-of-consecutive-values-fulfilling-condition-in-a-numpy-array
def contiguous_regions(condition):
    idx = []
    i = 0
    while i < len(condition):
        x1 = i + condition[i:].argmax()
        try:
            x2 = x1 + condition[x1:].argmin()
        except:
            x2 = x1 + 1
        if x1 == x2:
            if condition[x1] == True:
                x2 = len(condition)
            else:
                break
        idx.append( [x1,x2] )
        i = x2
    return idx

contiguous_regions(Y_crop>0.5)

In [ ]:
X[:,1]

In [ ]:
import re
remove_punc = re.compile('[\,\.\?\!]')
squash_spaces = re.compile('\s+')
def words(s):
    s = remove_punc.sub(' ', s)
    s = squash_spaces.sub(' ', s)
    return s.strip().lower()

sentences=dict(
    num=words("zero one two three four five six seven eight nine."),
    
# https://www.quora.com/Is-there-a-text-that-covers-the-entire-English-phonetic-range/answer/Sheetal-Srivastava-1
    qbf=words("That quick beige fox jumped in the air over each thin dog.  "+
              "Look out, I shout, for he's foiled you again, creating chaos."),
    shy=words("Are those shy Eurasian footwear, cowboy chaps, "+
              "or jolly earthmoving headgear?"),
    ate=words("The hungry purple dinosaur ate the kind, zingy fox, the jabbering crab, "+
              "and the mad whale and started vending and quacking."),
    suz=words("With tenure, Suzie'd have all the more leisure for yachting, "+
              "but her publications are no good."),
    tbh=words("Shaw, those twelve beige hooks are joined if I patch a young, gooey mouth."),
    
    #  https://en.wikipedia.org/wiki/The_North_Wind_and_the_Sun          #594
    #  http://videoweb.nie.edu.sg/phonetic/courses/aae103-web/wolf.html  #1111
)
sentences['num'] #.replace(' ', ',')

In [ ]:
def for_msft(prefixes):  # comma separated
    # https://www.microsoft.com/cognitive-services/en-us/speech-api
    return ' '.join([sentences[a] for a in prefixes.split(',')]).replace(' ', '\n') 
"""
This is the SSML that will be sent to the service:
<speak version="1.0" xmlns="http://www.w3.org/2001/10/synthesis" 
      xmlns:mstts="http://www.w3.org/2001/mstts" xml:lang="en-GB">
  <voice xml:lang="en-GB" name="Microsoft Server Speech Text to Speech Voice (en-GB, Susan, Apollo)">
zero
one
two
three
four
five
six
seven
eight
nine
  </voice>
</speak>
"""

# https://www.microsoft.com/cognitive-services/en-us/Speech-api/documentation/API-Reference-REST/BingVoiceOutput
a=for_msft('num')  # 49 long...
#a=for_msft('qbf,shy,ate,suz,tbh')  # 474 long...
print("length_in_chars=%d\n%s" % (len(a),a,))

In [ ]:
# sox english.au --rate 16000 --channels 1 --encoding signed-integer english.wav norm -3
# sox english.au --rate 16000 --channels 1 english.ogg norm -3

In [ ]:
import python_speech_features
import soundfile

#(sample_rate, samples) = scipy.io.wavfile.read(f)
samples, sample_rate = soundfile.read(f)

#sample_feat = python_speech_features.mfcc(samples, sample_rate, numcep=13, nfilt=26, appendEnergy=True)
#sample_feat = python_speech_features.mfcc(samples, sample_rate, numcep=28, nfilt=56, appendEnergy=True)

sample_feat, e = python_speech_features.fbank(samples,samplerate=sample_rate,winlen=0.025,winstep=0.01,nfilt=32,nfft=512,
      lowfreq=0,highfreq=None,preemph=0.25, winfunc=lambda x:np.hamming( x ))
#sample_feat, e = python_speech_features.fbank(samples,samplerate=sample_rate,winlen=0.025,winstep=0.01,nfilt=26,nfft=512,
#      lowfreq=0,highfreq=None,preemph=0.97, winfunc=lambda x:np.ones((x,)))

plt.imshow(np.log(sample_feat.T), interpolation='nearest',
    origin='lower',
    aspect='auto')

log_e = np.log(e)
log_e_hurdle = (log_e.max() - log_e.min())*0.25 + log_e.min()

plt.plot(log_e)

log_e_crop = np.where(log_e>log_e_hurdle, 1.0, 0.0)

#plt.plot(log_e_crop * 25)

# By smmothing, and applying a very low hurdle, we expand the crop area safely
log_e_crop_expanded = np.where(smooth(log_e_crop)>0.01, 1.0, 0.0)

plt.plot(log_e_crop_expanded * 25)

plt.show()
sample_feat.shape, e.shape, e[10]

In [ ]:
contiguous_regions(log_e_crop_expanded>0.5)